# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.113144e+02     1.549157e+00
 * time: 0.8443958759307861
     1     1.114771e+01     8.353108e-01
 * time: 2.3835818767547607
     2    -1.127352e+01     1.014645e+00
 * time: 3.1154069900512695
     3    -3.382039e+01     7.393873e-01
 * time: 4.210920810699463
     4    -4.731750e+01     5.349468e-01
 * time: 5.293068885803223
     5    -5.670542e+01     2.168858e-01
 * time: 6.3309409618377686
     6    -5.962867e+01     1.870439e-01
 * time: 7.0478198528289795
     7    -6.082333e+01     8.282118e-02
 * time: 7.779819011688232
     8    -6.133651e+01     5.052680e-02
 * time: 8.49852991104126
     9    -6.163144e+01     3.242310e-02
 * time: 9.2078378200531
    10    -6.179697e+01     3.626119e-02
 * time: 9.908308029174805
    11    -6.194255e+01     2.697668e-02
 * time: 10.63527798652649
    12    -6.203137e+01     1.753954e-02
 * time: 11.333834886550903
    13    -6.208745e+01     1.419807e-02
 * time: 12.070523977279663
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557749
    AtomicNonlocal      14.8522714
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336823

    total               -62.261666455864
